# 英文数据增强

> 参考：https://zhuanlan.zhihu.com/p/145521255

## 基于同义词典的替换

在这种方法中，我们从句子中随机取出一个单词，将其替换为对应的同义词。例如，我们可以使用英语的 WordNet 数据库来查找同义词，然后进行替换。WordNet 是一个人工维护的数据库，其中包含单词之间的关系。

对于如何使用，NLTK 提供了对 [WordNet](https://www.nltk.org/howto/wordnet.html) 的接口；我们还可以使用 [TextBlob API](https://textblob.readthedocs.io/en/dev/quickstart.html#wordnet-integration) 。此外，还有一个名为 [PPDB](http://paraphrase.org/#/download) 的数据库，其中包含数百万条同义词典，可以通过编程方式下载和使用。

## 基于 Word-Embeddings 的替换

在这种方法中，我们采用预先训练好的词向量，如 Word2Vec、GloVe、FastText，用向量空间中距离最近的单词替换原始句子中的单词。Jiao 等人在他们的论文 "TinyBert" 中使用了这种方法，以改进语言模型在下游任务上的泛化性；Wang 等人使用它来对 tweet 语料进行数据增强来学习主题模型。

例如，可以用三个向量空间中距离最近的单词替换原始句子中的单词，可以得到原始句子的三个变体。我们可以使用像 Gensim 包来完成这样的操作。在下面这个例子中，我们通过在 Tweet 语料上训练的词向量找到了单词 "awesome" 的同义词。

![](img/20200615135556.png)


In [2]:
import gensim.downloader as api

In [4]:
model = api.load('glove-twitter-25')  
model.most_similar('awesome', topn=5)

[('amazing', 0.9687871932983398),
 ('best', 0.9600659608840942),
 ('fun', 0.9331520795822144),
 ('fantastic', 0.9313924312591553),
 ('perfect', 0.9243415594100952)]

会得到 5 个最相似的单词以及对应的余弦相似度。

## 基于 Masked Language Model 的替换

像 BERT、ROBERTA 和 ALBERT 这样基于 Transformer 的模型已经使用 "Masked Language Modeling" 的方式，即模型要根据上下文来预测被 Mask 的词语，通过这种方式在大规模的文本上进行预训练。

Masked Language Modeling 同样可以用来做文本的数据增强。例如，我们可以使用一个预先训练好的 BERT 模型，然后对文本的某些部分进行 Mask，让 BERT 模型预测被 Mask 的词语。

![](img/20200615135759.png)

我们称这种方法叫 Mask Predictions。和之前的方法相比，这种方法生成的文本在语法上更加通顺，因为模型在进行预测的时候考虑了上下文信息。我们可以很方便的使用 HuggingFace 的 transfomers 库，通过设置要替换的词语并生成预测来做文本的数据增强。

![](img/20200615135841.png)

In [ ]:
from transformers import pipeline
nlp = pipeline('fill-mask')
nlp('This is <mask> cool')

然而，这种方法的一个需要注意的地方是如何确定要 Mask 文本的哪一部分，一般需要使用启发式的方法来决定 Mask 的地方，否则生成的文本可能无法保留原句的含义。

## 基于 TF-IDF 的替换

这种数据增强方法是 Xie 等人在 "Unsupervised Data Augmentation" 论文中提出来的。其基本思想是，TF-IDF 分数较低的单词不能提供信息，因此可以在不影响句子的基本真值标签的情况下替换它们。

![](20200615172044.png)

具体如何计算整个文档中单词的 TF-IDF 分数并选择最低的单词来进行替换，可以参考作者公开的[代码](https://link.zhihu.com/?target=https%3A//github.com/google-research/uda/blob/master/text/augmentation/word_level_augment.py)。

# Back Translation（回译）

# Text Surface Transformation

这些是使用正则表达式应用的简单模式匹配变换，Claude Coulombe 在他的论文中介绍了这些变换的方法。

在论文中，他给出了一个将动词由缩写形式转换为非缩写形式的例子，我们可以通过这个简单的方法来做文本的数据增强。

![](img/20200615172413.png)